<a href="https://colab.research.google.com/github/ozturkergin/ozturkergin/blob/main/TEFAS_PowerBI_Import_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  pip install bs4 --quiet

In [ ]:
pip install urllib3 --quiet

In [ ]:
pip install marshmallow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 774.3 kB/s eta 0:00:00


In [ ]:
!pip install pandas_ta --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
pip install free-proxy --quiet

  Preparing metadata (setup.py) ... done


In [ ]:
import requests
import pandas as pd
import math
import concurrent.futures
import time

from datetime import datetime, timedelta, date
from typing import Dict, List, Optional, Union
from marshmallow import Schema, fields, EXCLUDE, pre_load, post_load
from bs4 import BeautifulSoup
from fp.fp import FreeProxy

# Special thanks to https://github.com/burakyilmaz321

class InfoSchema(Schema):
    code = fields.String(data_key="FONKODU", allow_none=True)
    fonunvantip = fields.String(data_key="FONUNVANTIP", allow_none=True)
    date = fields.Date(data_key="TARIH", allow_none=True)
    price = fields.Float(data_key="FIYAT", allow_none=True)
    title = fields.String(data_key="FONUNVAN", allow_none=True)
    market_cap = fields.Float(data_key="PORTFOYBUYUKLUK", allow_none=True)
    number_of_shares = fields.Float(data_key="TEDPAYSAYISI", allow_none=True)
    number_of_investors = fields.Float(data_key="KISISAYISI", allow_none=True)

    @pre_load
    def pre_load_hook(self, input_data, **kwargs):
        seconds_timestamp = int(input_data["TARIH"]) / 1000
        input_data["TARIH"] = date.fromtimestamp(seconds_timestamp).isoformat()
        return input_data

    @post_load
    def post_load_hool(self, output_data, **kwargs):
        output_data = {f: output_data.setdefault(f) for f in self.fields}
        return output_data

    class Meta:
        unknown = EXCLUDE

class tefas_get:
    root_url = "https://www.tefas.gov.tr"
    info_endpoint = "/api/DB/BindHistoryInfo"
    concurrently = False
    use_Proxy = False
    fon_type = "YAT"
    proxies = None

    @staticmethod
    def get_combobox_items(url, select_id):
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception(f"Failed to fetch the URL: {response.status_code}")

        soup = BeautifulSoup(response.content, 'html.parser')
        select_element = soup.find('select', id=select_id)

        if not select_element:
            raise Exception(f"Select element with id '{select_id}' not found")

        options = select_element.find_all('option')
        options = list(filter(None, options))

        items = []
        for option in options:
            value = option.get('value')
            items.append(value)

        items.remove('')

        return items

    def fetch_info(self, fonunvantip, start_date_initial, end_date_initial):
        counter = 1
        start_date = start_date_initial
        end_date = end_date_initial
        range_date = end_date_initial - start_date_initial
        range_interval = 90
        info_schema = InfoSchema(many=True)
        info_result = pd.DataFrame()

        if range_date.days > range_interval :
            counter = range_date.days / range_interval
            counter = math.ceil(counter)
            end_date = start_date + timedelta(days=range_interval)

        while counter > 0:
            counter -= 1

            data = {
                    "fontip": self.fon_type,
                    "bastarih": self._parse_date(start_date),
                    "bittarih": self._parse_date(end_date),
                    "fonunvantip": fonunvantip,
                    "fonkod": "",
                  }

            info = self._do_post(data)
            info = info_schema.load(info)
            info = pd.DataFrame(info, columns=info_schema.fields.keys())
            #info['fonunvantip'] = fonunvantip.replace("(","").replace(")","").replace(" ","")
            info['fonunvantip'] = fonunvantip
            #info = info[info['price'] != 0]
            if not info.empty :
                info_result = pd.concat([info_result, info])
                info_result = info_result.reset_index(drop=True)
                info = info.reset_index(drop=True)

            if counter > 0 :
                start_date = end_date + timedelta(days=1)
                end_date = end_date + timedelta(days=range_interval)
                if end_date > end_date_initial :
                    end_date = end_date_initial

        return info_result

    def fetch_info_serial(self, fonunvantips, start_date_initial, end_date_initial):
        merged = pd.DataFrame()

        for fonunvantip in fonunvantips:
            info = self.fetch_info(fonunvantip, start_date_initial, end_date_initial)
            if not info.empty :
                merged = pd.concat([merged, info])
                print(f"{fonunvantip} - {len(info)} records added total records: {len(merged)} " )

        return merged

    def fetch_info_concurrently(self, fonunvantips, start_date_initial, end_date_initial):
        with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
            merged = pd.DataFrame()
            # Submit all tasks to the executor
            self.concurrently = True
            futures = {executor.submit(self.fetch_info, fonunvantip, start_date_initial, end_date_initial): fonunvantip for fonunvantip in fonunvantips}

            # Retrieve results as they complete
            for future in concurrent.futures.as_completed(futures):
                info = future.result()
                merged = pd.concat([merged, info])
                print(f"{future} - {len(info)} records added total records: {len(merged)} " )

            return merged

    def fetch(
        self,
        start: Union[str, datetime],
        end: Optional[Union[str, datetime]] = None,
        columns: Optional[List[str]] = None,
        unvantip: bool = False,
    ) -> pd.DataFrame:

        start_date_initial = datetime.strptime(start, "%Y-%m-%d")
        end_date_initial = datetime.strptime(end or start, "%Y-%m-%d")

        merged = pd.DataFrame()
        fonunvantips = [""]
        if unvantip :
            fonunvantips = self.get_combobox_items(url="https://www.tefas.gov.tr/TarihselVeriler.aspx", select_id="DropDownListFundTypeExplanationYAT")

        if self.use_Proxy :
            proxy_address = self.get_free_proxy()
            self.proxies = {"http": proxy_address, "https": proxy_address}
            print(self.proxies)
        else :
            self.proxies = None

        if self.concurrently :
            merged = self.fetch_info_concurrently(fonunvantips, start_date_initial, end_date_initial)
        else :
            merged = self.fetch_info_serial(fonunvantips, start_date_initial, end_date_initial)

        merged = merged[columns] if columns and not merged.empty else merged

        return merged

    def get_free_proxy(self):
        proxy_address = FreeProxy(timeout=1, rand=True, https=True).get()
        return proxy_address

    def _do_post(self, data: Dict[str, str]) -> Dict[str, str]:
        timestamp = int(time.time() * 1000)  # Get current timestamp in milliseconds
        headers = {
         "Connection": "keep-alive",
         "Cache-Control": "no-cache",
         "Pragma": "no-cache",
         "X-Requested-With": "XMLHttpRequest",
         "Sec-Fetch-Mode": "cors",
         "Sec-Fetch-Site": "same-origin",
         "Accept-Language": "tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7",
         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
         "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
         "Accept": "application/json, text/javascript, */*; q=0.01",
         "Origin": "https://www.tefas.gov.tr",
         "Referer": f"https://www.tefas.gov.tr/TarihselVeriler.aspx?timestamp={timestamp}" ,
         }

        response = requests.post(
             url=f"{self.root_url}/{self.info_endpoint}",
             data=data,
             proxies=self.proxies,
             headers=headers,
         )
        # Check the response status code and content
        if response.status_code != 200:
            print(f"Request failed with status code: {response.status_code}")
            print(f"Response content: {response.text}")
            return {}  # Return an empty dictionary if the request failed
        try:
            return response.json().get("data", {})
        except ValueError as e:
            print(f"Error decoding JSON response: {e}")
            print(f"Response content: {response.text}")
            return {}

    def _parse_date(self, date: Union[str, datetime]) -> str:
        if isinstance(date, datetime):
            formatted = datetime.strftime(date, "%d.%m.%Y")
        elif isinstance(date, str):
            try:
                parsed = datetime.strptime(date, "%Y-%m-%d")
            except ValueError as exc:
                raise ValueError(
                    "Date string format is incorrect. " "It should be `YYYY-MM-DD`"
                ) from exc
            else:
                formatted = datetime.strftime(parsed, "%d.%m.%Y")
        else:
            raise ValueError(
                "`date` should be a string like 'YYYY-MM-DD' "
                "or a `datetime.datetime` object."
            )
        return formatted

In [ ]:
tefas = tefas_get()

time_delta = 450
start_date_calc = date.today() - timedelta(days=time_delta)
date_start = start_date_calc.strftime("%Y-%m-%d")
date_end = date.today().strftime("%Y-%m-%d")

fetched_data = pd.DataFrame()
fetched_data = tefas.fetch(start=date_start, end=date_end, columns=["code", "date", "price", "market_cap", "number_of_shares", "number_of_investors"], unvantip=False)
fetched_data['date'] = pd.to_datetime(fetched_data['date'], errors='coerce')
fetched_data['date'].dt.strftime('%Y-%m-%d')
fetched_data['date'] = fetched_data['date'].dt.date
fetched_data['price'].astype(float,False)
fetched_data.rename(columns={'price': 'close'}, inplace=True)
fetched_data.rename(columns={'code': 'symbol'}, inplace=True)
fetched_data['market_cap'].astype(float,False)
fetched_data['number_of_shares'].astype(float,False)
fetched_data['number_of_investors'].astype(float,False)
fetched_data['market_cap_per_investors'] = fetched_data['market_cap'] / fetched_data['number_of_investors']
fetched_data[(fetched_data!=0)&(pd.isnull(fetched_data))]
fetched_data = fetched_data.sort_values(['symbol', 'date'])
fetched_data['open'] = fetched_data.groupby('symbol')['close'].shift(1)
fetched_data['high'] = fetched_data[['open', 'close']].max(axis=1)
fetched_data['low'] = fetched_data[['open', 'close']].min(axis=1)
fetched_data = fetched_data.dropna()
fetched_data.to_csv('tefas.csv', encoding='utf-8-sig', index=False)
fetched_data

In [ ]:
fetched_data=pd.read_csv('tefas.csv')
fetched_data['date'] = pd.to_datetime(fetched_data['date'])
fetched_data['qty'] = 100/fetched_data['close']
fetched_data['year'] = fetched_data['date'].dt.year
fetched_data['week_no'] = fetched_data['date'].dt.isocalendar().week.astype(str).str.zfill(2)
fetched_data['year_week'] = fetched_data['year'].astype(str) +'-'+ fetched_data['week_no'].astype(str)
fetched_data['day_of_week'] = fetched_data['date'].apply(lambda x: x.strftime('%A'))
idx = fetched_data.groupby(['symbol', 'year_week'])['date'].idxmax()
max_prices = fetched_data.loc[idx, ['symbol', 'year_week', 'close']]
max_prices = max_prices.rename(columns={'close': 'price_at_week_close'})
fetched_data = fetched_data.merge(max_prices, on=['symbol', 'year_week'], how='left')
fetched_data['valuation_at_week_close'] = fetched_data['price_at_week_close'] * fetched_data['qty']
fetched_data.to_csv('tefas_transformed.csv', encoding='utf-8-sig', index=False)

In [ ]:
fetched_data=pd.read_csv('tefas_transformed.csv')
fetched_data

In [ ]:
fetched_data_agg = tefas.fetch(start=date_end, end=date_end, columns=["code", "date", "price", "fonunvantip", "title"], unvantip=True)

In [ ]:
fon_table = fetched_data_agg.pivot_table(index=['title', 'code'], columns='fonunvantip', aggfunc='size', fill_value=0)
fon_table.reset_index(inplace=True)
fon_table = fon_table.replace(0, False)
fon_table = fon_table.replace(1, True)
fon_table.rename(columns={'code': 'symbol'}, inplace=True)
fon_table['symbol_with_title'] = fon_table['symbol'].astype(str) +' - '+ fon_table['title'].astype(str)
fon_table.to_csv('fon_table.csv', encoding='utf-8-sig', index=False)
fon_table